# Play with TensorBoard
> TensorBoard provides the visualization and tooling needed for machine learning experimentation

- toc: true
- badges: true
- author: Austin Chen
- categories: [tensorboard]

TensorBoard is TensorFlow's visualization toolkit, enabling ones to track metrics like loss and accuracy, visualize the model graph, view histograms of weights, biases, or other tensors as they change over time, and much more. 


In [ ]:
#collapse-hide

%load_ext tensorboard
import os
import numpy as np
import pandas as pd
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

pd.options.display.max_columns=25

In [ ]:
df = pd.read_csv("kc_house_data.csv")
df.shape

(21613, 21)

In [ ]:
df.head().T

,0,1,2,3,4
id,7129300520,6414100192,5631500400,2487200875,1954400510
date,20141013T000000,20141209T000000,20150225T000000,20141209T000000,20150218T000000
price,221900,538000,180000,604000,510000
bedrooms,3,3,2,4,3
bathrooms,1,2.25,1,3,2
sqft_living,1180,2570,770,1960,1680
sqft_lot,5650,7242,10000,5000,8080
floors,1,2,1,1,1
waterfront,0,0,0,0,0
view,0,0,0,0,0


In [ ]:
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [ ]:
df['year'] = pd.to_numeric(df['date'].str.slice(0,4))
df['month'] = pd.to_numeric(df['date'].str.slice(4,6))
df['day'] = pd.to_numeric(df['date'].str.slice(6,8))

df.drop(['id', 'date'], axis="columns", inplace=True)
df.head().T

,0,1,2,3,4
price,221900.0000,538000.000,180000.0000,604000.0000,510000.0000
bedrooms,3.0000,3.000,2.0000,4.0000,3.0000
bathrooms,1.0000,2.250,1.0000,3.0000,2.0000
sqft_living,1180.0000,2570.000,770.0000,1960.0000,1680.0000
sqft_lot,5650.0000,7242.000,10000.0000,5000.0000,8080.0000
floors,1.0000,2.000,1.0000,1.0000,1.0000
waterfront,0.0000,0.000,0.0000,0.0000,0.0000
view,0.0000,0.000,0.0000,0.0000,0.0000
condition,3.0000,3.000,3.0000,5.0000,3.0000
grade,7.0000,7.000,6.0000,7.0000,8.0000


In [ ]:
n = df.shape[0]
ids = np.random.permutation(n)
train_ids = ids[:int(n * .6)]
valid_ids = ids[int(n * .4) : int(n * .8)]
test_ids = ids[int(n * .8):]

train_data = df.loc[train_ids]
valid_data = df.loc[valid_ids]
test_data = df.loc[test_ids]

In [ ]:
train_valid_data = pd.concat([train_data, valid_data])
mean = train_valid_data.mean()
std = train_valid_data.std()
train_data = (train_data - mean) / std
valid_data = (valid_data - mean) / std

In [ ]:
train_x = np.array(train_data.drop('price', axis='columns')).astype('float32')
train_y = np.array(train_data['price']).astype('float32')

valid_x = np.array(valid_data.drop('price', axis='columns')).astype('float32')
valid_y = np.array(valid_data['price']).astype('float32')

In [ ]:
train_x.shape, valid_x.shape

((12967, 21), (8645, 21))

In [ ]:
model = tf.keras.Sequential(name='model-1')
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(21,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.summary()

Model: "model-1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1408      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,633
Trainable params: 5,633
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(tf.keras.optimizers.Adam(0.001),
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
log_dir = "logs/model_1/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('models/best-model-1.h5',
                                                monitor='val_mean_absolute_error',
                                                save_best_only=True,
                                                mode='min')

In [ ]:
#collapse_output
history = model.fit(train_x, train_y,
                    batch_size=64,
                    epochs=300,
                    validation_data=(valid_x, valid_y),
                    callbacks=[tensorboard_callback,
                               checkpoint_callback])

Epoch 1/300
203/203 [==============================] - 4s 7ms/step - loss: 0.4115 - mean_absolute_error: 0.4102 - val_loss: 0.2153 - val_mean_absolute_error: 0.2894
Epoch 2/300
203/203 [==============================] - 1s 4ms/step - loss: 0.2147 - mean_absolute_error: 0.2880 - val_loss: 0.1866 - val_mean_absolute_error: 0.2655
Epoch 3/300
203/203 [==============================] - 1s 4ms/step - loss: 0.1913 - mean_absolute_error: 0.2684 - val_loss: 0.1700 - val_mean_absolute_error: 0.2589
Epoch 4/300
203/203 [==============================] - 1s 4ms/step - loss: 0.1584 - mean_absolute_error: 0.2487 - val_loss: 0.1553 - val_mean_absolute_error: 0.2431
Epoch 5/300
203/203 [==============================] - 1s 4ms/step - loss: 0.1488 - mean_absolute_error: 0.2400 - val_loss: 0.1401 - val_mean_absolute_error: 0.2347
Epoch 6/300
203/203 [==============================] - 1s 4ms/step - loss: 0.1440 - mean_absolute_error: 0.2348 - val_loss: 0.1443 - val_mean_absolute_error: 0.2353
Epoch 7/30

In [ ]:
#hide
!tensorboard dev upload \
  --logdir logs/model_1 \
  --name "Blog: Play with TensorBoard" \
  --description "Simple comparison of several hyperparameters" \
  --one_shot

2021-03-03 09:30:50.127962: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/MVQyms8BSVym5wG2ETdDyA/

[2021-03-03T09:30:52] Started scanning logdir.
[2021-03-03T09:30:53] Total uploaded: 1200 scalars, 0 tensors, 1 binary objects (33.1 kB)
[2021-03-03T09:30:53] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/MVQyms8BSVym5wG2ETdDyA/


In [ ]:
#collapse
from  IPython import display
display.IFrame(
    src="https://tensorboard.dev/experiment/MVQyms8BSVym5wG2ETdDyA/",
    width = "100%",
    height="800px")